This assignment is inspired by: 

- https://www.kaggle.com/code/carlmcbrideellis/an-introduction-to-xgboost-regression
- https://www.kaggle.com/code/dansbecker/xgboost/notebook

In this assignment we will apply XGBoost Regression techniques to predict house prices, based on the famous Kaggle Dataset https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques

Step 1 is to download the dataset.

In [2]:
#=========================================================================
# load up the libraries
#=========================================================================
import pandas  as pd
import numpy   as np
import xgboost as xgb
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
import xgboost as xgb

#=========================================================================
# read in the data
#=========================================================================
train_data = pd.read_csv('train.csv',index_col=0)
test_data  = pd.read_csv('test.csv',index_col=0)

In [4]:
train_data.shape

(1460, 80)

In [5]:
test_data.shape

(1459, 79)

### <center style="background-color:Gainsboro; width:60%;">Feature selection</center>
The purpose of feature selection, as the name suggests, is to only model the most pertinent and important features, thus reducing the computational overhead, and also to alleviate the [curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality). The following are a number of notebooks covering techniques to achieve said goal, all of which use the House Prices data as an example:

* [Feature selection using the Boruta-SHAP package](https://www.kaggle.com/carlmcbrideellis/feature-selection-using-the-boruta-shap-package)
* [Recursive Feature Elimination (RFE) example](https://www.kaggle.com/carlmcbrideellis/recursive-feature-elimination-rfe-example)
* [House Prices: Permutation Importance example](https://www.kaggle.com/carlmcbrideellis/house-prices-permutation-importance-example)
* [Feature importance using the LASSO](https://www.kaggle.com/carlmcbrideellis/feature-importance-using-the-lasso)

In this assignment, we shall use all of the numerical columns, and ignore the categorical features. To encode the categorical features one can use for example [pandas.get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html). 

Our first task is to do Feature Exploration and Selection. 

In [23]:
# use only numerical predictors
train_predictors = train_data.select_dtypes(exclude=['object'])
test_predictors = test_data.select_dtypes(exclude=['object'])

# Impute missing values with mean
train_predictors.fillna(train_predictors.mean(), inplace=True)
test_predictors.fillna(test_predictors.mean(), inplace=True)


### <center style="background-color:Gainsboro; width:60%;">Feature engineering</center>
As mentioned, one aspect of feature engineering is the creation of new features out of existing features. A simple example would be to create a new feature which is the sum of the number of bathrooms in the house:

In [24]:
# Step 4: Preparing the data for modeling
# Select the target variable
y = train_data['SalePrice']

# Split the data into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(train_predictors, y, test_size=0.2, random_state=42)


In [55]:
print("Shape of y:", y.shape)


Shape of y: (1460,)


In [56]:
print("Shape of X_train:", X_train.shape)


Shape of X_train: (1168, 42)


In [57]:
print("Shape of X_test:", X_test.shape)


Shape of X_test: (292, 42)


In [58]:
# Calculate the number of bathrooms
#X_train['n_bathrooms'] = X_train['BsmtFullBath'] + (0.5 * X_train['BsmtHalfBath']) + X_train['FullBath'] + (0.5 * X_train['HalfBath'])
#X_test['n_bathrooms'] = X_test['BsmtFullBath'] + (0.5 * X_test['BsmtHalfBath']) + X_test['FullBath'] + (0.5 * X_test['HalfBath'])

# Calculate the area including basement
#X_train['area_with_basement'] = X_train['GrLivArea'] + X_train['TotalBsmtSF']
#X_test['area_with_basement'] = X_test['GrLivArea'] + X_test['TotalBsmtSF']

# Step 5: Feature engineering
# Calculate the number of bathrooms
for df in [X_train, X_test]:
    df['n_bathrooms'] = df['BsmtFullBath'] + (0.5 * df['BsmtHalfBath']) + df['FullBath'] + (0.5 * df['HalfBath'])

# Calculate the area including basement
for df in [X_train, X_test]:
    df['area_with_basement'] = df['GrLivArea'] + df['TotalBsmtSF']


Your next task is to apply some feature engineering to prepare for using the XGBoost Estimator to predict house prices.

In [59]:
# Calculate Total Square Foot of the house

for df in [train_data, test_data]:
    df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']

for df in [train_data, test_data]:
    df['Age'] = df['YrSold'] - df['YearBuilt']

#X_train['TotalSF'] = X_train['TotalBsmtSF'] + X_train['1stFlrSF'] + X_train['2ndFlrSF']
#X_test['TotalSF'] = X_test['TotalBsmtSF'] + X_test['1stFlrSF'] + X_test['2ndFlrSF']

# Calculate Age of the house at the time of sale
#X_train['Age'] = X_train['YrSold'] - X_train['YearBuilt']
#X_test['Age'] = X_test['YrSold'] - X_test['YearBuilt']


In [63]:
print( 'X_train shape:', X_train.shape)

X_train shape: (1168, 42)


In [64]:
print( 'X_test shape:', X_test.shape)

X_test shape: (292, 42)


In [66]:
print("Shape of y_train:", y_train.shape)


Shape of y_train: (1168,)


For more on this fascinating aspect may I recommend the free on-line book ["*Feature Engineering and Selection: A Practical Approach for Predictive Models*"](http://www.feat.engineering/) by Max Kuhn and Kjell Johnson.
### <center style="background-color:Gainsboro; width:60%;">XGBoost estimator</center>
Note that for this competition we use the RMSLE evaluation metric, rather than the default metric, which for regression is the RMSE. For more on the peculiarities of the RMSLE see the Appendix below.

In [ ]:
#=========================================================================
# XGBoost regression: 
# Parameters: 
# n_estimators  "Number of gradient boosted trees. Equivalent to number 
#                of boosting rounds."
# learning_rate "Boosting learning rate (also known as “eta”)"
# max_depth     "Maximum depth of a tree. Increasing this value will make 
#                the model more complex and more likely to overfit." 
#=========================================================================
#regressor=xgb.XGBRegressor(eval_metric='rmsle')

#=========================================================================
# exhaustively search for the optimal hyperparameters
#=========================================================================
#from sklearn.model_selection import GridSearchCV
# set up our search grid
#param_grid = {"max_depth":    [4, 5],
              #"n_estimators": [500, 600, 700],
              #"learning_rate": [0.01, 0.015]}

Can you use grid search to find the optimal hyper parameters?

In [65]:
# Inside the GridSearchCV code (before fitting the model)
print("Shape of dataset used for fitting the model:", X_train.shape, y_train.shape)


Shape of dataset used for fitting the model: (1168, 42) (1168,)


In [28]:
# Use GridSearchCV to find the best parameters
from sklearn.model_selection import GridSearchCV

# Define our XGBoost regressor
xgb_regressor = xgb.XGBRegressor(eval_metric='rmsle')

# Set up our search grid
param_grid = {"max_depth": [4, 5],
              "n_estimators": [500, 600, 700],
              "learning_rate": [0.01, 0.015]}

# Define GridSearchCV
grid_search = GridSearchCV(estimator=xgb_regressor,
                           param_grid=param_grid,
                           cv=5, verbose=2, n_jobs=1)

# Run the Grid Search
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=500; total time=   1.6s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=500; total time=   1.0s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=500; total time=   1.0s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=500; total time=   1.0s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=500; total time=   1.0s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=600; total time=   1.2s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=600; total time=   1.2s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=600; total time=   1.2s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=600; total time=   1.2s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=600; total time=   1.2s
[CV] END ..learning_rate=0.01, max_depth=4, n_estimators=700; total time=   1.4s
[CV] END ..learning_rate=0.01, max_depth=4, n_es

In [30]:
# Print the best parameters
print("Best parameters found: ", grid_search.best_params_)

Best parameters found:  {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 600}


In [31]:
# Print the best score found
print("Best score found: ", grid_search.best_score_)

Best score found:  0.9959983929747972


Now, can you setup a XGBoost Regressor object using your hyperparameters and fit it?

In [32]:
# Define the XGBoost regressor with the optimal hyperparameters
optimal_xgb_regressor = xgb.XGBRegressor(learning_rate=0.01, max_depth=4, n_estimators=600, eval_metric='rmsle')

# Fit the model to the training data
optimal_xgb_regressor.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmsle', feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=600, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [33]:
from sklearn.metrics import mean_squared_log_error

# Make predictions on the validation data
y_pred = optimal_xgb_regressor.predict(X_test)

# Compute RMSLE on the validation data
rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))

print("RMSLE on validation data: ", rmsle)

RMSLE on validation data:  0.017178577380050315


Finally, can you run it on your test set?

In [46]:
# Run the model on the test.csv*****
from sklearn.metrics import mean_squared_log_error

# Train the XGBoost model with the optimal hyperparameters
optimal_xgb_regressor.fit(X_train, y_train)

# Make predictions on the validation data
y_pred = optimal_xgb_regressor.predict(X_test)

# Compute RMSLE on the validation data
rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))

print("RMSLE on validation data: ", rmsle)



RMSLE on validation data:  0.017178577380050315


In [42]:
# Check the column names in X_train and test_predictors
print("Columns in X_train: ", X_train.columns)
print("Columns in test_predictors: ", test_predictors.columns)

Columns in X_train:  Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold', 'SalePrice', 'TotalSF', 'Age', 'HasGarage',
       'n_bathrooms', 'area_with_basement'],
      dtype='object')
Columns in test_predictors:  Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', '

In [43]:
# Find the missing columns in test_predictors
missing_columns = set(X_train.columns) - set(test_predictors.columns)

In [44]:
# Add the missing columns to test_predictors with default values (e.g., 0)
for col in missing_columns:
    test_predictors[col] = 0

In [45]:
# Reorder test_predictors columns to match X_train
test_predictors = test_predictors[X_train.columns]

In [47]:
# Make prediction on test data
test_pred = optimal_xgb_regressor.predict(test_predictors)

# Print the predicted house prices for the test data
print("Predicted house prices for test data: ", test_pred)

Predicted house prices for test data:  [39016.98 39016.98 39016.98 ... 39016.98 39016.98 39016.98]


In [ ]:
Can you score your solution offline and see how it does?

In [48]:
from sklearn.metrics import mean_squared_log_error
from math import sqrt

# Make predictions on the test data
test_pred = optimal_xgb_regressor.predict(X_test)

# Calculate the RMSLE
rmsle = sqrt(mean_squared_log_error(y_test, test_pred))

print("RMSLE on test data: ", rmsle)

RMSLE on test data:  0.017178577380050315


In [38]:
X_test.shape

(292, 42)

In [39]:
y_true.shape

(1459,)

In [49]:
# read in the ground truth file
solution = pd.read_csv('sample_submission.csv') 
y_true = solution["SalePrice"]

# Make predictions on the test data
predictions = optimal_xgb_regressor.predict(X_test)

# Calculate the RMSLE
RMSLE = np.sqrt(mean_squared_log_error(y_true, predictions))
print("The score is %.5f" % RMSLE)


ValueError: Found input variables with inconsistent numbers of samples: [1459, 292]

Finally, use the below block to prepare your submission

In [ ]:
output = pd.DataFrame({"Id":test_data.index, "SalePrice":predictions})
output.to_csv('submission.csv', index=False)

### <center style="background-color:Gainsboro; width:60%;">Feature importance</center>
Let us also take a very quick look at the feature importance too:

In [ ]:
from xgboost import plot_importance
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams.update({'font.size': 16})

fig, ax = plt.subplots(figsize=(12,6))
plot_importance(regressor, max_num_features=8, ax=ax)
plt.show();

Where here the `F score` is a measure "*...based on the number of times a variable is selected for splitting, weighted by the squared improvement to the model as a result of each split, and averaged over all trees*." [1] 

Note that these importances are susceptible to small changes in the training data, and it is much better to make use of ["GPU accelerated SHAP values"](https://www.kaggle.com/carlmcbrideellis/gpu-accelerated-shap-values-jane-street-example), incorporated with version 1.3 of XGBoost.

Can you follow the above guide use SHAP values instead of F Score?

In [1]:
# code here

### <center style="background-color:Gainsboro; width:60%;">Appendix: The RMSLE evaluation metric</center>
From the competition [evaluation page](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview/evaluation) we see that the metric we are using is the root mean squared logarithmic error (RMSLE), which is given by

$$ {\mathrm {RMSLE}}\,(y, \hat y) = \sqrt{ \frac{1}{n} \sum_{i=1}^n \left(\log (1 + \hat{y}_i) - \log (1 + y_i)\right)^2} $$

where $\hat{y}_i$ is the predicted value of the target for instance $i$, and $y_i$
is the actual value of the target for instance $i$.

It is important to note that, unlike the RMSE, the RMSLE is asymmetric; penalizing much more the underestimated predictions than the overestimated predictions. For example, say the correct value is $y_i = 1000$, then underestimating by 600 is almost twice as bad as overestimating by 600:

In [ ]:
def RSLE(y_hat,y):
    return np.sqrt((np.log1p(y_hat) - np.log1p(y))**2)

print("The RMSLE score is %.3f" % RSLE( 400,1000) )
print("The RMSLE score is %.3f" % RSLE(1600,1000) )

The asymmetry arises because 

$$ \log (1 + \hat{y}_i) - \log (1 + y_i) =  \log \left( \frac{1 + \hat{y}_i}{1 + y_i} \right) $$

so we are essentially looking at ratios, rather than differences such as is the case of the RMSE. We can see the form that this asymmetry takes in the following plot, again using 1000 as our ground truth value:

In [ ]:
plt.rcParams["figure.figsize"] = (7, 4)
x = np.linspace(5,4000,100)
plt.plot(x, RSLE(x,1000))
plt.xlabel('prediction')
plt.ylabel('RMSLE')
plt.show()